In [2]:
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql import functions as F

now = datetime.now()
spark = SparkSession.builder.appName("first_challenge").getOrCreate()
sc = spark.sparkContext
black_friday_dates = ["2020-11-27","2019-11-29","2018-11-23","2017-11-24","2016-11-25"]

df = spark.read.csv("./clientes_pedidos.csv",header=True)

# Correcting datatype of column data_pedido
df_with_time = df.withColumn("date_pedido",F.from_unixtime("data_pedido","yyyy-MM-dd").cast('date'))

# Adding age
df_with_age  = df_with_time.withColumn('age',(F.months_between(F.lit(now),
                                        F.col('data_nascimento_cliente'),True)/12).cast('int'))

# Buyers who bougth on black friday
buyers_from_bf = df_with_age.where(F.col('date_pedido').isin(black_friday_dates))

# Join pedido with date
joined_date_df =  buyers_from_bf.withColumn("joined_pedido",F.array("codigo_pedido","date_pedido"))

# Aggregated values by rules set on test
joined_agg = joined_date_df.where('age > 30').groupBy(
    'codigo_cliente','age').agg(
    F.collect_list("joined_pedido").alias("lista_pedidos"),
    F.count("joined_pedido").alias("numero_pedidos")).where("numero_pedidos > 2")


joined_agg.show()

+--------------------+---+--------------------+--------------+
|      codigo_cliente|age|       lista_pedidos|numero_pedidos|
+--------------------+---+--------------------+--------------+
|057ae5d7ef3fcdd74...| 67|[[413131491beb7ab...|             3|
|20bd2aa7874c75558...| 52|[[f629e03c1df7976...|             2|
|2ee219338175cf9a3...| 56|[[a196b59841c1b9e...|             2|
|3795da4cfbd4e112e...| 59|[[63c434ce079842c...|             2|
|44a92ee8614d92d9c...| 38|[[c53dc1fa839192c...|             2|
|5900d20f6d2db0a90...| 31|[[eccde45a3a207be...|             2|
|621bfc6f65835fa32...| 65|[[45018ca4e1401be...|             3|
|78b9e69c2edca7234...| 69|[[0246a03dd2b9423...|             2|
|97116589c0613591d...| 38|[[52f2e930774722a...|             2|
|a18a317475d9675a6...| 38|[[bdf338beb59b501...|             2|
|b4bd9982468dec686...| 58|[[bf4019845b6a2a2...|             2|
|be598b79ed5740dc6...| 57|[[3de95561763439c...|             2|
|c4a135c3ffb30c3f4...| 37|[[54a5414bc651310...|        